<h1>Abgabe</h1>

In [1]:
!pip install yfinance
!pip install pandas


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Daten von yfinance herunterladen
data_aapl = yf.download("AAPL", start="2010-01-01", end="2022-12-31")
data_msft = yf.download("MSFT", start="2010-01-01", end="2022-12-31")

# Hull Moving Average (HMA) berechnen
def HMA(data, window):
    half_length = int(window / 2)
    sqrt_length = int(np.sqrt(window))
    WMA_half = data.rolling(window=half_length).apply(lambda x: np.average(x, weights=range(1, half_length + 1)), raw=True)
    WMA_full = data.rolling(window=window).apply(lambda x: np.average(x, weights=range(1, window + 1)), raw=True)
    raw_HMA = 2 * WMA_half - WMA_full
    HMA = raw_HMA.rolling(window=sqrt_length).apply(lambda x: np.average(x, weights=range(1, sqrt_length + 1)), raw=True)
    return HMA

# Berechnung der gleitenden Durchschnitte und HMA für AAPL und MSFT
def calculate_moving_averages(data):
    data['SMA_20'] = data['Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()
    data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['HMA_20'] = HMA(data['Close'], 20)
    data['HMA_50'] = HMA(data['Close'], 50)
    return data

data_aapl = calculate_moving_averages(data_aapl)
data_msft = calculate_moving_averages(data_msft)

# Funktion zur Generierung von Kauf- und Verkaufssignalen basierend auf HMA
def generate_hma_signals(data, short_window, long_window, volume_threshold=1000000, price_threshold=50):
    data['Signal'] = 0

    short_hma = data[f'HMA_{short_window}']

    buy_signal_condition = (data['Close'] > short_hma) & (data['Volume'] > volume_threshold) & (data['Close'] > price_threshold)
    sell_signal_condition = (data['Close'] < short_hma) & (data['Volume'] > volume_threshold) & (data['Close'] > price_threshold)

    data.loc[buy_signal_condition, 'Signal'] = 1
    data.loc[sell_signal_condition, 'Signal'] = -1

    # Reduzierung von Rauschen durch Einführung einer Mindestanzahl von Tagen zwischen Signalen
    data['Position'] = 0
    last_signal_date = None
    min_days_between_signals = 5  # Mindestanzahl von Tagen zwischen Signalen

    for i in range(len(data)):
        if data['Signal'].iloc[i] != 0:
            if last_signal_date is None or (data.index[i] - last_signal_date).days >= min_days_between_signals:
                data.loc[data.index[i], 'Position'] = data['Signal'].iloc[i]
                last_signal_date = data.index[i]

    return data

# Überarbeitete Funktion zur Generierung von Kauf- und Verkaufssignalen an Tief- und Hochpunkten
def generate_hma_signals_peak_trough(data, short_window, long_window, volume_threshold=1000000, price_threshold=50):
    data['Signal'] = 0

    short_hma = data[f'HMA_{short_window}']

    # Bedingung für Kaufsignale: Kurs unterhalb des kurzen HMA und innerhalb eines bestimmten Schwellenwerts zum Tiefpunkt
    buy_signal_condition = (data['Close'] > short_hma) & (data['Volume'] > volume_threshold) & (data['Close'] > price_threshold)
    # Bedingung für Verkaufssignale: Kurs oberhalb des kurzen HMA und innerhalb eines bestimmten Schwellenwerts zum Hochpunkt
    sell_signal_condition = (data['Close'] < short_hma) & (data['Volume'] > volume_threshold) & (data['Close'] > price_threshold)

    data.loc[buy_signal_condition, 'Signal


SyntaxError: unterminated string literal (detected at line 69) (771179976.py, line 69)